In [1]:
setwd("C:/Users/jojo/OneDrive/Dokumente/Uni/BACHELORARBEIT/Analysis/tpj---r-analysis")

#install.packages('XLConnect')
#install.packages('ggpubr')
#install.packages('plyr')
#install.packages('dplyr')
#install.packages('reshape2')
#install.packages('tidyverse')
#install.packages('car')
#install.packages('ggplot2')
#install.packages('nortest')
#install.packages('fingerprint')
#install.packages('MASS')
#install.packages('ez')

library(XLConnect)
library(ggpubr)
library(plyr)
library(dplyr)
library(reshape2)
library(tidyverse)
library(car)
library(ggplot2)
library(nortest)
library(fingerprint)
library(MASS)
library(ez)

Warning message:
"package 'XLConnect' was built under R version 3.5.3"Loading required package: XLConnectJars
Warning message:
"package 'XLConnectJars' was built under R version 3.5.3"XLConnect 0.2-15 by Mirai Solutions GmbH [aut],
  Martin Studer [cre],
  The Apache Software Foundation [ctb, cph] (Apache POI),
  Graph Builder [ctb, cph] (Curvesapi Java library)
http://www.mirai-solutions.com
https://github.com/miraisolutions/xlconnect
Warning message:
"package 'ggpubr' was built under R version 3.5.3"Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.5.3"Loading required package: magrittr
Warning message:
"package 'magrittr' was built under R version 3.5.3"Warning message:
"package 'plyr' was built under R version 3.5.3"
Attaching package: 'plyr'

The following object is masked from 'package:ggpubr':

    mutate

Warning message:
"package 'dplyr' was built under R version 3.5.3"
Attaching package: 'dplyr'

The following objects are masked

## Pipeline for analysing the data for my Bachelor Thesis

The first part is written in Python. This sections will be explained and written down in Markdown, so they are not runnable in here. Use a Python IDE to test them.

Also, a few things I did in the console without a script: The outlier detection, characterization and t-tests. 

To explain all three of them shortly: 

For the outlier detection, I calculated the standard deviation and means of each variable and looped through the values. If a value was three standard deviations away from the mean, I printed the value out. The outlier were then marked in all_data_final.xlsx.

For characterization, I used mean() and sem()

For the t-tests, I used t.test()


The data was contained in a file named "all_data_final.xlsx". To extract the data from the .mat files, I used a Python script as well. As this is a long script, spread out over several files, this is not included here. You can find it on my Github (github.com/gibbonjojo).

## Behavioural data
First up is the transformation of the data to prepare for all further analysis steps.

We begin with importing the modules and loading the data

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.stats import shapiro, skew, probplot, boxcox

# load the data
os.chdir(r"C:\Users\jojou\OneDrive\Dokumente\Uni\BACHELORARBEIT\Analysis")
d = pd.read_excel("all_data_final.xlsx")

# define a dataframe
data = {"rt_posner": d.loc[:, ["group", "Participant_ID", "age", "sex", "rt_valid_pre", "rt_valid_post", "rt_invalid_pre", "rt_invalid_post"]].loc[d["outlier_posner"]==0].loc[d["Exclude"]==0], 
        "correct_posner": d.loc[:, ["group", "Participant_ID", "age", "sex", "correct_valid_pre", "correct_valid_post", "correct_invalid_pre", "correct_invalid_post"]].loc[d["outlier_posner"]==0].loc[d["Exclude"]==0], 
        "eff_posner": d.loc[:, ["group", "Participant_ID", "age", "sex", "Inv_eff_sc_valid_pre", "Inv_eff_sc_valid_post", "Inv_eff_sc_invalid_pre", "Inv_eff_sc_invalid_post"]].loc[d["outlier_posner"]==0].loc[d["Exclude"]==0], 
        "rt_director": d.loc[:, ("group", "Participant_ID", "age", "sex", "rt_pt_pre", "rt_pt_post", "rt_npt_pre", "rt_npt_post")].loc[d["outlier_director"]==0].loc[d["Exclude"]==0], 
        "acc_director": d.loc[:, ("group", "Participant_ID", "age", "sex", "acc_pt_pre", "acc_pt_post", "acc_npt_pre", "acc_npt_post")].loc[d["outlier_director"]==0].loc[d["Exclude"]==0], 
        "eff_director": d.loc[:, ["group", "Participant_ID", "age", "sex", "Inv_eff_sc_npt_pre", "Inv_eff_sc_npt_post", "Inv_eff_sc_pt_pre", "Inv_eff_sc_pt_post"]].loc[d["outlier_director"]==0].loc[d["Exclude"]==0], 
        "TPJ_posner": d.loc[:, ["group", "Participant_ID", "age", "sex", "TPJ_valid_pre", "TPJ_valid_post", "TPJ_invalid_pre", "TPJ_invalid_post"]].loc[d["outlier_posner_TPJ"]==0].loc[d["Exclude"]==0], 
        "TPJ_director": d.loc[:, ("group", "Participant_ID", "age", "sex", "TPJ_pt_pre", "TPJ_npt_pre", "TPJ_npt_post", "TPJ_pt_post")].loc[d["outlier_director_TPJ"]==0].loc[d["Exclude"]==0]}


```

Next, I defined a custom melt function, since the ANOVA later needed the data to be melted

```python
# takes 4 arguments: The dataframe to be melted, a precursor, which is the variable (e.g. rt or acc) and both conditions (valid/invalid or pt/npt)
def melting(df, prec, cond1, cond2):
    # first melts the whole dataframe
    melted = pd.melt(df, id_vars=("group", "Participant_ID", "age", "sex")) # melt the data, except four columns, using pandas melt function. See pd.melt for explanation
    
    # define the names of the values in the variable column
    one_pre = f"{prec}_{cond1}_pre"
    one_post = f"{prec}_{cond1}_post"
    two_pre = f"{prec}_{cond2}_pre"
    two_post = f"{prec}_{cond2}_post"
    
    # append two columns to the dataframe - time and condition
    melted["time"] = pd.Series(np.array("x" for i in range(len(melted.values))), index=melted.index)
    melted["condition"] = pd.Series(np.array("x" for i in range(len(melted.values))), index=melted.index)

    # fill the two new columns with the time and condition values
    melted["time"].loc[melted["variable"] == one_pre] = "pre"
    melted["time"].loc[melted["variable"] == two_pre] = "pre"
    melted["time"].loc[melted["variable"] == one_post] = "post"
    melted["time"].loc[melted["variable"] == two_post] = "post"
    
    melted["condition"].loc[melted["variable"] == one_pre] = cond1
    melted["condition"].loc[melted["variable"] == two_pre] = cond2
    melted["condition"].loc[melted["variable"] == one_post] = cond1
    melted["condition"].loc[melted["variable"] == two_post] = cond2

    return melted



# drop NAs
for i, j in zip(data, datamelted):
    data[i].dropna(inplace=True)
    datamelted[j].dropna(inplace=True)
    
    
    
# For the Boxcox transformation:
# Split the data in Up and Downregulation group
data_up = {n:df.loc[df["group"]==1] for n, df in data.items()}
data_down = {n:df.loc[df["group"]==2] for n, df in data.items()}

# boxcox transformation on the data
for n, df in data_up.items():
    if n != "TPJ_posner" and n != "TPJ_director" and n != "acc_director" and n != "correct_posner":
        for dat in data_up[n]:
            if dat not in ["group", "age", "sex", "Participant_ID"]:
                print(n, ", ", dat)
                print(f"Before Transformation: {shapiro(df[dat])}")
                df[dat] = boxcox(df[dat])[0]
                print(f"After Transformation: {shapiro(df[dat])}")

for n, df in data_down.items():
    if n != "TPJ_posner" and n != "TPJ_director" and n != "acc_director" and n != "correct_posner":
        for dat in data_down[n]:
            if dat not in ["group", "age", "sex", "Participant_ID"]:
                print(n, ", ", dat)
                print(f"Before Transformation: {shapiro(df[dat])}")
                df[dat] = boxcox(df[dat])[0]
                print(f"After Transformation: {shapiro(df[dat])}")
  
# connect up and downregulation group again.
data_boxcox = {n:df_up.append(df_down, ignore_index=True) for (n, df_up), (_, df_down) in zip(data_up.items(), data_down.items())}


# Melt the data
datamelted2_boxcox = {"rt_posner": melting(data_boxcox["rt_posner"], "rt", "valid", "invalid"),
              "correct_posner": melting(data_boxcox["correct_posner"], "correct", "valid", "invalid"),
              "eff_posner": melting(data_boxcox["eff_posner"], "Inv_eff_sc", "valid", "invalid"),
              "rt_director": melting(data_boxcox["rt_director"], "rt", "npt", "pt"),
              "acc_director": melting(data_boxcox["acc_director"], "acc", "npt", "pt"),
              "eff_director": melting(data_boxcox["eff_director"], "Inv_eff_sc", "npt", "pt"),
              "TPJ_posner": melting(data_boxcox["TPJ_posner"], "TPJ", "valid", "invalid"),
              "TPJ_director": melting(data_boxcox["TPJ_director"], "TPJ", "npt", "pt"),
             }

# save the data to an excel file
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

# open the Workbook
wb = openpyxl.Workbook()

# create the worksheets
for x in datamelted2_boxcox.keys():
    if x == "rt_posner":
        sheet1 = wb.active
        sheet1.title = x
        continue
    wb.create_sheet(x)
    
# fill the worksheets
for x, s in zip(datamelted2_boxcox.values(), wb.worksheets):
    for row in dataframe_to_rows(x, index=True, header=True):
        s.append(row)
        
path = r"C:\Users\jojou\OneDrive\Dokumente\Uni\BACHELORARBEIT\Analysis\boxcox_data_melted.xlsx"
wb.save(path)
    
```

### This file can now be opened in R for analysis


First, define a function to open the excel file and to prepare the dataframes

In [2]:
getMeltedData = function(){
  setwd(PATH_TO_DATA)
  
  # load the data
  wb = loadWorkbook("boxcox_data_melted.xlsx")

  
  rt_posner = readWorksheet(wb, sheet="rt_posner") 
  correct_posner = readWorksheet(wb, sheet="correct_posner")
  eff_posner= readWorksheet(wb, sheet="eff_posner")
  
  rt_director = readWorksheet(wb, sheet="rt_director")
  acc_director = readWorksheet(wb, sheet="acc_director")
  eff_director = readWorksheet(wb, sheet="eff_director")
  
  # transform the accuracies to ranks
  correct_posner$value <- rank(correct_posner$value)
  acc_director$value <- rank(acc_director$value)
  
  
  TPJ_posner = readWorksheet(wb, sheet="TPJ_posner")
  TPJ_director = readWorksheet(wb, sheet="TPJ_director")
  
  returnlist = list(rt_posner=rt_posner, correct_posner=correct_posner, eff_posner=eff_posner, TPJ_posner=TPJ_posner, 
                    rt_director=rt_director, acc_director=acc_director, eff_director=eff_director, TPJ_director=TPJ_director) 
  return(returnlist)
}

In [3]:
# load the data
anovalist <- getMeltedData()

Now, I looped through the dataframes and performed ANOVAs for each of them.
The output contains all results for the ANOVAs

In [4]:
for (name in names(anovalist)){
  data = anovalist[name][[1]]
    
  data.complete <- data[complete.cases(data), ]

  # Perform the complete ANOVA. add aov = TRUE to get an aov object for post-hoc analysis
  data.ezanova <- ezANOVA(data = data.complete, 
                          dv = value, 
                          wid = Participant_ID, 
                          type = 3, 
                          within = c(time, condition), 
                          between = group, 
                          detailed = TRUE)
  print(name)
  print("ANOVA FOR BOTH GROUPS")
  print(data.ezanova)
  
  # Perform simple ANOVAs for the seperate groups to investigate within-group effects
  data.up <- data.complete %>% filter(group == 1)
  data.up.ezanova <- ezANOVA(data = data.up, 
                             dv = value,
                             wid = Participant_ID,
                             type = 3,
                             detailed = TRUE,
                             within = c(condition, time))
  print(name)
  print("ANOVA FOR UPREGULATION GROUP")
  print(data.up.ezanova)
  
  data.down <- data.complete %>% filter(group == 2)
  data.down.ezanova <- ezANOVA(data = data.down, 
                             dv = value,
                             wid = Participant_ID,
                             type = 3,
                             detailed = TRUE,
                             within = c(condition, time))
  print(name)
  print("ANOVA FOR DOWNREGULATION GROUP")
  print(data.down.ezanova)

  print("                   ")
}

Warning message:
"Converting "Participant_ID" to factor for ANOVA."Warning message:
"Converting "time" to factor for ANOVA."Warning message:
"Converting "condition" to factor for ANOVA."Warning message:
""group" will be treated as numeric."Warning message:
"Data is unbalanced (unequal N per group). Make sure you specified a well-considered value for the type argument to ezANOVA()."

[1] "rt_posner"
[1] "ANOVA FOR BOTH GROUPS"
$ANOVA
                Effect DFn DFd          SSn         SSd         F            p
1          (Intercept)   1  27 43.259028634 19.80497397 58.974769 2.930018e-08
2                group   1  27  2.085742242 19.80497397  2.843480 1.032688e-01
3                 time   1  27  0.370854511  0.89318081 11.210577 2.407690e-03
5            condition   1  27  1.624540397  1.96696145 22.299670 6.425230e-05
4           group:time   1  27  0.242428399  0.89318081  7.328378 1.162540e-02
6      group:condition   1  27  0.219337194  1.96696145  3.010788 9.411481e-02
7       time:condition   1  27  0.013011656  0.08189396  4.289874 4.802166e-02
8 group:time:condition   1  27  0.003096102  0.08189396  1.020768 3.213059e-01
  p<.05          ges
1     * 0.6553798624
2       0.0839915853
3     * 0.0160419016
5     * 0.0666572442
4     * 0.0105452074
6       0.0095503734
7     * 0.0005716891
8       0.0001360918



Warning message:
"Converting "Participant_ID" to factor for ANOVA."Warning message:
"Converting "condition" to factor for ANOVA."Warning message:
"Converting "time" to factor for ANOVA."

[1] "rt_posner"
[1] "ANOVA FOR UPREGULATION GROUP"
$ANOVA
          Effect DFn DFd         SSn         SSd          F            p p<.05
1    (Intercept)   1  19 190.8233487 11.30157678 320.808653 2.341858e-13     *
2      condition   1  19   5.4693376  1.70325573  61.011046 2.386264e-07     *
3           time   1  19   0.4035135  0.78983079   9.706834 5.693833e-03     *
4 condition:time   1  19   0.0340977  0.06320358  10.250310 4.697199e-03     *
          ges
1 0.932295366
2 0.282986481
3 0.028294140
4 0.002454491



Warning message:
"Converting "Participant_ID" to factor for ANOVA."Warning message:
"Converting "condition" to factor for ANOVA."Warning message:
"Converting "time" to factor for ANOVA."

...

[1] "eff_director"
[1] "ANOVA FOR UPREGULATION GROUP"
$ANOVA
          Effect DFn DFd          SSn          SSd            F            p
1    (Intercept)   1  20 1.201090e+01 7.114149e-04 3.376623e+05 9.358966e-44
2      condition   1  20 1.993287e-05 6.643526e-05 6.000691e+00 2.363577e-02
3           time   1  20 1.214489e-04 6.525164e-05 3.722477e+01 5.804425e-06
4 condition:time   1  20 8.456047e-06 4.572919e-05 3.698315e+00 6.882259e-02
  p<.05         ges
1     * 0.999926003
2     * 0.021934052
3     * 0.120213086
4       0.009424015



Warning message:
"Converting "Participant_ID" to factor for ANOVA."Warning message:
"Converting "condition" to factor for ANOVA."Warning message:
"Converting "time" to factor for ANOVA."

[1] "eff_director"
[1] "ANOVA FOR DOWNREGULATION GROUP"
$ANOVA
          Effect DFn DFd          SSn          SSd            F            p
1    (Intercept)   1  11 6.848525e+00 5.494317e-04 1.371122e+05 6.997636e-24
2      condition   1  11 1.489117e-05 4.541686e-05 3.606653e+00 8.407568e-02
3           time   1  11 4.429073e-05 4.731986e-05 1.029585e+01 8.324562e-03
4 condition:time   1  11 6.341744e-06 3.976487e-05 1.754292e+00 2.121971e-01
  p<.05         ges
1     * 0.999900436
2       0.021370046
3     * 0.060987696
4       0.009213968

[1] "                   "


Warning message:
"Converting "Participant_ID" to factor for ANOVA."Warning message:
"Converting "time" to factor for ANOVA."Warning message:
"Converting "condition" to factor for ANOVA."Warning message:
""group" will be treated as numeric."Warning message:
"Data is unbalanced (unequal N per group). Make sure you specified a well-considered value for the type argument to ezANOVA()."

ERROR: Error in ezANOVA_main(data = data, dv = dv, wid = wid, within = within, : One or more cells is missing data. Try using ezDesign() to check your data.


### Plots
For easier handling, a function is defined:

This function takes a few arguments: First of all both conditions (valid/invalid or npt/pt). Then the variables (e.g. rt_valid_pre, rt_valid_post,...), then the plot title, the y-axis label and then the range of the y-axis.

If you want to replicate this piece of code, you also have to define a variable called "plot_path", which is the path, where you want to save the plot

Also a function is defined to summarize the data and calculate means and deviations, which I found [here](http://tiramisutes.github.io/2015/09/21/line-ggplots.html)

In [ ]:
# Helperfunction to summarize the data and calculate means and deviations
data_summary <- function(data, varname, groupnames){
  require(plyr)
  summary_func <- function(x, col){
    c(mean = mean(as.double(x[[col]]), na.rm = TRUE),
      sd = sd(x[[col]], na.rm = TRUE),
      sem = sd(x[[col]], na.rm = TRUE)/sqrt(length(x[[col]])))
  }
  data_sum <- ddply(data, groupnames, .fun = summary_func, varname)
  data_sum <- plyr::rename(data_sum, c("mean" = varname))
  return(data_sum)
}

# Function for Plot
getPlot <- function(cond1, cond2, var1, var2, var3, var4, var5, var6, var7, var8, title, ylab=values, ranges){
  
  # Put the data into a data.frame
  df = data.frame(group = factor(rep(c("Up", "Down"), each = length(var1)*4), levels = c("Up", "Down")),
                     session = factor(rep(c("Pre", "Post"), 4, each = length(var1)), levels = c("Pre", "Post")),
                     condition = factor(rep(c(cond1, cond2), 2, each = length(var1)*2), levels = c(cond1, cond2)),
                     values = c(var1, var2, var3, var4,
                                var5, var6, var7, var8))
  
  # summarize the data
  data <- data_summary(df, varname="values", groupnames=c("group", "session", "condition"))
  
  # create the plot
  plot <- ggplot(data, aes(condition, values, fill=session)) +
    geom_col(position = position_dodge())+
    facet_grid(. ~ group)+
    ggtitle(title) +
    labs(y = ylab)+
    coord_cartesian(ylim=ranges)+
    theme(plot.title = element_text(size = 30), 
          legend.position="bottom", 
          axis.title.x = element_text(size = 24),
          axis.title.y = element_text(size = 24),
          strip.text.x = element_text(size = 24),
          legend.text = element_text(size = 20),
          legend.title = element_text(size = 24),
          axis.text.x = element_text(size = 20),
          axis.text.y = element_text(size = 20))+
    geom_errorbar(aes(ymin=values-sem, ymax=values+sem), 
                  width = 0.2, 
                  position = position_dodge(.9))+
    ggsave(unit="cm", width=27, height=23, filename = file.path(plot_path,paste(title,'.pdf',sep='')))+
    ggsave(unit="cm", width=27, height=23, filename = file.path(plot_path,paste(title,'.png',sep='')))

  
  # return the plot
  return(plot)
}

Now I can call the function to create the plots:

In [ ]:
# RT Posner
getPlot1("Valid", "Invalid", 
        up_posner$rt_valid_pre, up_posner$rt_valid_post, up_posner$rt_invalid_pre, up_posner$rt_invalid_post,
        down_posner$rt_valid_pre, down_posner$rt_valid_post, down_posner$rt_invalid_pre, down_posner$rt_invalid_post,
        "Reaction Time (Posner)", "rt [secs]", c(0.35, 0.60))
        
# Inverse Efficiency Score Posner
getPlot1("Valid", "Invalid", 
        up_posner$Inv_eff_sc_valid_pre, up_posner$Inv_eff_sc_valid_post, up_posner$Inv_eff_sc_invalid_pre, up_posner$Inv_eff_sc_invalid_post,
        down_posner$Inv_eff_sc_valid_pre, down_posner$Inv_eff_sc_valid_post, down_posner$Inv_eff_sc_invalid_pre, down_posner$Inv_eff_sc_invalid_post,
        "Inverse Efficiency Score (Posner)", "score", c(0.35, 0.60))
                                
# RT Director
getPlot1("Non-perspective taking", "Perspective taking", 
         up_director$rt_npt_pre, up_director$rt_npt_post, up_director$rt_pt_pre, up_director$rt_pt_post,
         down_director$rt_npt_pre, down_director$rt_npt_post, down_director$rt_pt_pre, down_director$rt_pt_post,
         "Reaction Time (Director)", "rt [secs]", c(3,4))
                             
# Accuracy Director
getPlot1("Non-perspective taking", "Perspective taking", 
         up_director$acc_npt_pre, up_director$acc_npt_post, up_director$acc_pt_pre, up_director$acc_pt_post,
         down_director$acc_npt_pre, down_director$acc_npt_post, down_director$acc_pt_pre, down_director$acc_pt_post,
         "Accuracy (Director)", "correct", c(0.85, 1.01))

## Regulation Performance
First define functions for loading the data

In [5]:
# Load the data and melt it
getMeltedPerformance = function(){
  setwd(PATH_TO_DATA)
  
  # load the data
  wb = loadWorkbook("all_data_final.xlsx")
  data = readWorksheet(wb, sheet="Performance_md_run")
  data = data %>% filter(Exclude==0)
  
  data = melt(data, id.vars=c("Participant_ID", "group"), measure.vars=c("Run1", "Run2", "Run3","Run4", "Run5", "Run6","Run7", "Run8", "Run9","Run10", "Run11", "Run12"))
  
  
  return(data)
}

# Just load the data
getPerformance = function(){
  setwd(PATH_TO_DATA)
  
  # load the data
  wb = loadWorkbook("all_data_final.xlsx")
  data = readWorksheet(wb, sheet="Performance_md_run")
  data = data %>% filter(Exclude==0)
  return(data)
}

Get the data

In [6]:
data.melted = getMeltedPerformance()
data = getPerformance()

### Perform the ANOVA

In [7]:
data.ezanova <- ezANOVA(data = data.melted, dv = value, wid = Participant_ID, type = 3, within = variable, between = group, detailed = TRUE)
print(data.ezanova)

Warning message:
"Converting "Participant_ID" to factor for ANOVA."Warning message:
""group" will be treated as numeric."Warning message:
"Data is unbalanced (unequal N per group). Make sure you specified a well-considered value for the type argument to ezANOVA()."

$ANOVA
          Effect DFn DFd       SSn      SSd         F           p p<.05
1    (Intercept)   1  34 1275.6878 3508.309 12.363048 0.001263623     *
2          group   1  34  198.2338 3508.309  1.921139 0.174755018      
3       variable  11 374  227.0678 6306.961  1.224093 0.268813933      
4 group:variable  11 374  294.8574 6306.961  1.589537 0.099509705      
         ges
1 0.11502053
2 0.01979665
3 0.02261105
4 0.02916456

$`Mauchly's Test for Sphericity`
          Effect          W           p p<.05
3       variable 0.03915457 0.006441151     *
4 group:variable 0.03915457 0.006441151     *

$`Sphericity Corrections`
          Effect       GGe     p[GG] p[GG]<.05      HFe     p[HF] p[HF]<.05
3       variable 0.6332554 0.2902815           0.813123 0.2797122          
4 group:variable 0.6332554 0.1395557           0.813123 0.1179690          



### Calculate Correlation with behavioral data [Slope Trial1 - Trial 72 against pre vs post]

In [13]:
# load behavioral data
behav = loadWorkbook("all_data_final.xlsx")
behav = readWorksheet(behav, sheet="task_data")

# exclude participants
behav = behav %>% filter(Exclude == 0)

# filter by regulation
behav.up = behav %>% filter(group == 1)
behav.down = behav %>% filter(group == 2)

# filter by task and exclude participants specifically for the seperate tasks
behav.posner.up = dplyr::select(behav.up %>% filter(outlier_posner == 0), c(Participant_ID, rt_valid_pre:Diff_inv_eff_sc_invalid)) 
behav.director.up = dplyr::select(behav.up %>% filter(outlier_director == 0), c(Participant_ID, rt_npt_pre:Diff_Inv_eff_sc_pt)) 

behav.posner.down = dplyr::select(behav.down %>% filter(outlier_posner == 0), c(Participant_ID, rt_valid_pre:Diff_inv_eff_sc_invalid)) 
behav.director.down = dplyr::select(behav.down %>% filter(outlier_director == 0), c(Participant_ID, rt_npt_pre:Diff_Inv_eff_sc_pt)) 

# Calculate differences
behav.posner.up.diff <- data.frame(rt_valid = behav.posner.up$rt_valid_post - behav.posner.up$rt_valid_pre,
                          rt_invalid = behav.posner.up$rt_invalid_post - behav.posner.up$rt_invalid_pre,
                          correct_valid = behav.posner.up$correct_valid_post - behav.posner.up$correct_valid_pre,
                          correct_invalid = behav.posner.up$correct_invalid_post - behav.posner.up$correct_invalid_pre,
                          eff_valid = behav.posner.up$Inv_eff_sc_valid_post - behav.posner.up$Inv_eff_sc_valid_pre,
                          eff_invalid = behav.posner.up$Inv_eff_sc_invalid_post - behav.posner.up$Inv_eff_sc_invalid_pre)

behav.posner.down.diff <- data.frame(rt_valid = behav.posner.down$rt_valid_post - behav.posner.down$rt_valid_pre,
                                   rt_invalid = behav.posner.down$rt_invalid_post - behav.posner.down$rt_invalid_pre,
                                   correct_valid = behav.posner.down$correct_valid_post - behav.posner.down$correct_valid_pre,
                                   correct_invalid = behav.posner.down$correct_invalid_post - behav.posner.down$correct_invalid_pre,
                                   eff_valid = behav.posner.down$Inv_eff_sc_valid_post - behav.posner.down$Inv_eff_sc_valid_pre,
                                   eff_invalid = behav.posner.down$Inv_eff_sc_invalid_post - behav.posner.down$Inv_eff_sc_invalid_pre)

behav.director.up.diff <- data.frame(rt_npt = behav.director.up$rt_npt_post - behav.director.up$rt_npt_pre,
                                     rt_pt = behav.director.up$rt_pt_post - behav.director.up$rt_pt_pre,
                                     acc_npt = behav.director.up$acc_npt_post - behav.director.up$acc_npt_pre,
                                     acc_pt = behav.director.up$acc_pt_post - behav.director.up$acc_pt_pre,
                                     eff_npt = behav.director.up$Inv_eff_sc_npt_post - behav.director.up$Inv_eff_sc_npt_pre,
                                     eff_pt = behav.director.up$Inv_eff_sc_pt_post - behav.director.up$Inv_eff_sc_pt_pre)

behav.director.down.diff <- data.frame(rt_npt = behav.director.down$rt_npt_post - behav.director.down$rt_npt_pre,
                                     rt_pt = behav.director.down$rt_pt_post - behav.director.down$rt_pt_pre,
                                     acc_npt = behav.director.down$acc_npt_post - behav.director.down$acc_npt_pre,
                                     acc_pt = behav.director.down$acc_pt_post - behav.director.down$acc_pt_pre,
                                     eff_npt = behav.director.down$Inv_eff_sc_npt_post - behav.director.down$Inv_eff_sc_npt_pre,
                                     eff_pt = behav.director.down$Inv_eff_sc_pt_post - behav.director.down$Inv_eff_sc_pt_pre)


Warning message:
"Error detected in cell AD37 - Value is not available"Warning message:
"Error detected in cell AE37 - Value is not available"Warning message:
"Error detected in cell AG37 - Value is not available"Warning message:
"Error detected in cell BH6 - Value is not available"Warning message:
"Error detected in cell BI6 - Value is not available"Warning message:
"Error detected in cell BJ6 - Value is not available"Warning message:
"Error detected in cell BK5 - Value is not available"Warning message:
"Error detected in cell BL5 - Value is not available"Warning message:
"Error detected in cell BM5 - Value is not available"

In [14]:
# Prepare Trial data
data.trials <- loadWorkbook("all_data_final.xlsx")
forExcludes <- readWorksheet(data.trials, sheet="task_data") %>% filter(Exclude==0)
data.trials <- readWorksheet(data.trials, sheet="Performance_md_trial") %>% filter(Exclude==0)

data.trials.posner.up <- data.trials %>% filter(forExcludes$outlier_posner==0) %>% filter(group == 1)
data.trials.posner.down <- data.trials %>% filter(forExcludes$outlier_posner==0) %>% filter(group == 2)
data.trials.director.up <- data.trials %>% filter(forExcludes$outlier_director==0) %>% filter(group == 1)
data.trials.director.down <- data.trials %>% filter(forExcludes$outlier_director==0) %>% filter(group == 2)


trials.posner.up <- data.frame()
trials.posner.up <- t(data.trials.posner.up[1:length(data.trials.posner.up$Trial1),8:79])

trials.posner.down <- data.frame()
trials.posner.down <- t(data.trials.posner.down[1:length(data.trials.posner.down$Trial1),8:79])

trials.director.up <- data.frame()
trials.director.up <- t(data.trials.director.up[1:length(data.trials.director.up$Trial1),8:79])

trials.director.down <- data.frame()
trials.director.down <- t(data.trials.director.down[1:length(data.trials.director.down$Trial1),8:79])

# for every participant, calculate their slope of Trials
slopes.posner.up = c()
for(i in c(1:length(data.trials.posner.up$Trial1))){
  slopes.posner.up[i] <- coef(lm(trials.posner.up[1:72, i] ~ c(1:72)))[2]
}

slopes.posner.down = c()
for(i in c(1:length(data.trials.posner.down$Trial1))){
  slopes.posner.down[i] <- coef(lm(trials.posner.down[1:72, i] ~ c(1:72)))[2]
}

slopes.director.up = c()
for(i in c(1:length(data.trials.director.up$Trial1))){
  slopes.director.up[i] <- coef(lm(trials.director.up[1:72, i] ~ c(1:72)))[2]
}

slopes.director.down = c()
for(i in c(1:length(data.trials.director.down$Trial1))){
  slopes.director.down[i] <- coef(lm(trials.director.down[1:72, i] ~ c(1:72)))[2]
}

Warning message:
"Error detected in cell AD37 - Value is not available"Warning message:
"Error detected in cell AE37 - Value is not available"Warning message:
"Error detected in cell AG37 - Value is not available"Warning message:
"Error detected in cell BH6 - Value is not available"Warning message:
"Error detected in cell BI6 - Value is not available"Warning message:
"Error detected in cell BJ6 - Value is not available"Warning message:
"Error detected in cell BK5 - Value is not available"Warning message:
"Error detected in cell BL5 - Value is not available"Warning message:
"Error detected in cell BM5 - Value is not available"

In [15]:
# Correlations

for (n in names(behav.posner.up.diff)){
  print(n)
  print(cor.test(behav.posner.up.diff[[n]], slopes.posner.up))
}

for (n in names(behav.posner.down.diff)){
  print(n)
  print(cor.test(behav.posner.down.diff[[n]], slopes.posner.down))
}

for (n in names(behav.director.up.diff)){
  print(n)
  print(cor.test(behav.director.up.diff[[n]], slopes.director.up))
}

for (n in names(behav.director.down.diff)){
  print(n)
  print(cor.test(behav.director.down.diff[[n]], slopes.director.down))
}


[1] "rt_valid"

	Pearson's product-moment correlation

data:  behav.posner.up.diff[[n]] and slopes.posner.up
t = 1.6904, df = 20, p-value = 0.1065
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.0799638  0.6746115
sample estimates:
      cor 
0.3535645 

[1] "rt_invalid"

	Pearson's product-moment correlation

data:  behav.posner.up.diff[[n]] and slopes.posner.up
t = 1.7618, df = 20, p-value = 0.09338
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.06513942  0.68265074
sample estimates:
      cor 
0.3665353 

[1] "correct_valid"

	Pearson's product-moment correlation

data:  behav.posner.up.diff[[n]] and slopes.posner.up
t = 0.063599, df = 20, p-value = 0.9499
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.4098458  0.4332309
sample estimates:
       cor 
0.01421972 

[1] "correct_invalid"

	Pearson's product-moment correlation

data:  behav.posne

### Plots
First, we calculated the means for the plots:

In [ ]:
mean_md = c(mean(data$Run1), mean(data$Run2), mean(data$Run3), mean(data$Run4), mean(data$Run5), mean(data$Run6), mean(data$Run7), mean(data$Run8), mean(data$Run9), mean(data$Run10), mean(data$Run11), mean(data$Run12))
mean_md.up = c(mean(data$Run1[data$group==1]), mean(data$Run2[data$group==1]), mean(data$Run3[data$group==1]), mean(data$Run4[data$group==1]), mean(data$Run5[data$group==1]), mean(data$Run6[data$group==1]), mean(data$Run7[data$group==1]), mean(data$Run8[data$group==1]), mean(data$Run9[data$group==1]), mean(data$Run10[data$group==1]), mean(data$Run11[data$group==1]), mean(data$Run12[data$group==1]))
mean_md.down = c(mean(data$Run1[data$group==2]), mean(data$Run2[data$group==2]), mean(data$Run3[data$group==2]), mean(data$Run4[data$group==2]), mean(data$Run5[data$group==2]), mean(data$Run6[data$group==2]), mean(data$Run7[data$group==2]), mean(data$Run8[data$group==2]), mean(data$Run9[data$group==2]), mean(data$Run10[data$group==2]), mean(data$Run11[data$group==2]), mean(data$Run12[data$group==2]))
Runs = c(1:12)
means = data.frame(Runs, mean_md, mean_md.up, mean_md.down)

And on to the plots:

In [ ]:
# The first plot which contains both groups and their linear regression line
ggplot()+
  #up
  geom_point(data = means, aes(x=Runs, y=mean_md.up), col = "blue") +
  stat_smooth(data = means, aes(x=Runs, y=mean_md.up), col = "blue") +
  stat_smooth(data = means, aes(x=Runs, y=mean_md.up), method = lm, se = FALSE, col = "blue") +
  #down
  geom_point(data = means, aes(x=Runs, y=mean_md.down), col = "red") +
  stat_smooth(data = means, aes(x=Runs, y=mean_md.down), col = "red") +
  stat_smooth(data = means, aes(x=Runs, y=mean_md.down), method = lm, se = FALSE, col = "red") +
  #config
  coord_cartesian(xlim = c(1, 12))+ 
  ggtitle("Regulation performance")+
  ylab("Regulation - Baseline (SD)")+
  scale_x_discrete(limits=seq(1,12))+
  theme(plot.title = element_text(size = 28), 
        axis.title.x = element_text(size = 22),
        axis.title.y = element_text(size = 22),
        strip.text.x = element_text(size = 22),
        axis.text.x = element_text(size = 18),
        axis.text.y = element_text(size = 18))+
  ggsave(unit="cm", width=27, height=15, filename = file.path(plot_path,paste("RegulationPerformance",'.pdf',sep='')))+
  ggsave(unit="cm", width=27, height=15, filename = file.path(plot_path,paste("RegulationPerformance",'.png',sep='')))


# This plot contains the upregulation group including boxplots
ggplot()+
  geom_boxplot(aes(x = 1, y = data$Run1[data$group==1]))+
  geom_boxplot(aes(x = 2, y = data$Run2[data$group==1]))+
  geom_boxplot(aes(x = 3, y = data$Run3[data$group==1]))+
  geom_boxplot(aes(x = 4, y = data$Run4[data$group==1]))+
  geom_boxplot(aes(x = 5, y = data$Run5[data$group==1]))+
  geom_boxplot(aes(x = 6, y = data$Run6[data$group==1]))+
  geom_boxplot(aes(x = 7, y = data$Run7[data$group==1]))+
  geom_boxplot(aes(x = 8, y = data$Run8[data$group==1]))+
  geom_boxplot(aes(x = 9, y = data$Run9[data$group==1]))+
  geom_boxplot(aes(x = 10, y = data$Run10[data$group==1]))+
  geom_boxplot(aes(x = 11, y = data$Run11[data$group==1]))+
  geom_boxplot(aes(x = 12, y = data$Run12[data$group==1]))+
  geom_smooth(data = means, aes(x = Runs, y = mean_md.up),method =lm, se = FALSE, color = "red") +
  coord_cartesian(xlim = c(1, 12), ylim = c(-18,38))+ 
  ggtitle("Upregulation group")+
  ylab("TPJ activation")+
  xlab("Neurofeedback Runs")+
  theme_grey(base_size = 14)+
  scale_x_discrete(limits=seq(1,12))+
  theme(plot.title = element_text(size = 14), 
        axis.title.x = element_text(size = 11),
        axis.title.y = element_text(size = 11),
        strip.text.x = element_text(size = 11),
        axis.text.x = element_text(size = 9),
        axis.text.y = element_text(size = 9))+
  ggsave(unit="cm", width=14, height=8, filename = file.path(plot_path,paste("Upregulation",'.pdf',sep='')))+
  ggsave(unit="cm", width=14, height=8, filename = file.path(plot_path,paste("Upregulation",'.png',sep='')))


# This plot contains the downregulation group including boxplots
ggplot()+
  geom_boxplot(aes(x = 1, y = data$Run1[data$group==2]))+
  geom_boxplot(aes(x = 2, y = data$Run2[data$group==2]))+
  geom_boxplot(aes(x = 3, y = data$Run3[data$group==2]))+
  geom_boxplot(aes(x = 4, y = data$Run4[data$group==2]))+
  geom_boxplot(aes(x = 5, y = data$Run5[data$group==2]))+
  geom_boxplot(aes(x = 6, y = data$Run6[data$group==2]))+
  geom_boxplot(aes(x = 7, y = data$Run7[data$group==2]))+
  geom_boxplot(aes(x = 8, y = data$Run8[data$group==2]))+
  geom_boxplot(aes(x = 9, y = data$Run9[data$group==2]))+
  geom_boxplot(aes(x = 10, y = data$Run10[data$group==2]))+
  geom_boxplot(aes(x = 11, y = data$Run11[data$group==2]))+
  geom_boxplot(aes(x = 12, y = data$Run12[data$group==2]))+
  geom_smooth(data = means, aes(x = Runs, y = mean_md.down),method =lm, se = FALSE, color = "red")+
  coord_cartesian(xlim = c(1, 12), ylim = c(-8,13))+ 
  ggtitle("Downregulation group")+
  ylab("TPJ activation")+
  xlab("Neurofeedback Runs")+
  theme_grey(base_size = 14)+
  scale_x_discrete(limits=seq(1,12))+
  theme(plot.title = element_text(size = 14), 
        axis.title.x = element_text(size = 11),
        axis.title.y = element_text(size = 11),
        strip.text.x = element_text(size = 11),
        axis.text.x = element_text(size = 9),
        axis.text.y = element_text(size = 9))+
  ggsave(unit="cm", width=14, height=8, filename = file.path(plot_path,paste("Downregulation",'.pdf',sep='')))+
  ggsave(unit="cm", width=14, height=8, filename = file.path(plot_path,paste("Downregulation",'.png',sep='')))
